In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
df = pd.read_csv('../data/archive/sales_data_sample.csv', encoding='utf-8', encoding_errors='replace')

In [5]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [6]:
df.describe()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,QTR_ID,MONTH_ID,YEAR_ID,MSRP
count,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.00000,2823.000000
mean,10258.725115,35.092809,83.658544,6.466171,3553.889072,2.717676,7.092455,2003.81509,100.715551
std,92.085478,9.741443,20.174277,4.225841,1841.865106,1.203878,3.656633,0.69967,40.187912
min,10100.000000,6.000000,26.880000,1.000000,482.130000,1.000000,1.000000,2003.00000,33.000000
25%,10180.000000,27.000000,68.860000,3.000000,2203.430000,2.000000,4.000000,2003.00000,68.000000
50%,10262.000000,35.000000,95.700000,6.000000,3184.800000,3.000000,8.000000,2004.00000,99.000000
75%,10333.500000,43.000000,100.000000,9.000000,4508.000000,4.000000,11.000000,2004.00000,124.000000
max,10425.000000,97.000000,100.000000,18.000000,14082.800000,4.000000,12.000000,2005.00000,214.000000


In [47]:
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

In [17]:
# Monthly Sales
monthly_sales_2004 = (
    df[(df['ORDERDATE'] >= '2004-01-01') & (df['ORDERDATE'] <= '2004-12-31')]
    .assign(YearMonth=lambda x: x['ORDERDATE'].dt.to_period('M'))
    .groupby('YearMonth')['SALES']
    .sum()
    .reset_index(name='TotalSales')
)

In [18]:
print(monthly_sales_2004)

   YearMonth  TotalSales
0    2004-01   316577.42
1    2004-02   311419.53
2    2004-03   205733.73
3    2004-04   206148.12
4    2004-05   273438.39
5    2004-06   286674.22
6    2004-07   327144.09
7    2004-08   461501.27
8    2004-09   320750.91
9    2004-10   552924.25
10   2004-11  1089048.01
11   2004-12   372802.66


In [59]:
# Average Daily Sales Per Month
sales_day_month = (
    df[(df['ORDERDATE']>= '2004-1-1') & (df['ORDERDATE'] <= '2004-12-31')][['ORDERDATE']].reset_index(drop=True)
)
print(sales_day_month)

      ORDERDATE
0    2004-01-15
1    2004-02-20
2    2004-04-05
3    2004-05-18
4    2004-06-28
...         ...
1340 2004-10-14
1341 2004-10-29
1342 2004-11-10
1343 2004-11-21
1344 2004-12-02

[1345 rows x 1 columns]


In [56]:
print(df['ORDERDATE'].min())

2003-01-06 00:00:00


In [23]:
df3 = pd.DataFrame({
    'A_1': [10, 20, 30],
    'A_2': [1,  2,  3],
    'B_1': [5,  6,  7]
})

grouped = df3.groupby(lambda x: x[0], axis=1).sum()
print(grouped)

    A  B
0  11  5
1  22  6
2  33  7


/tmp/ipykernel_6099/2092464948.py:7: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df3.groupby(lambda x: x[0], axis=1).sum()


In [44]:
df.groupby(pd.Grouper(key='ORDERDATE', freq='M'))['SALES'].sum().reset_index().agg(
    df[(df['ORDERDATE'] >= '2004-1-1') & (df['ORDERDATE'] <= '2004-12-31')]
)    


/tmp/ipykernel_6099/505381377.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df.groupby(pd.Grouper(key='ORDERDATE', freq='M'))['SALES'].sum().reset_index()


,ORDERDATE,SALES
0,2003-01-31,129753.60
1,2003-02-28,140836.19
2,2003-03-31,174504.90
3,2003-04-30,201609.55
4,2003-05-31,192673.11
5,2003-06-30,168082.56
6,2003-07-31,187731.88
7,2003-08-31,197809.30
8,2003-09-30,263973.36
9,2003-10-31,568290.97
